# Implementing Basic RAG QA with Langchain
This notebook demonstrates how to implement a basic Retrieval-Augmented Generation (RAG) chain using our FAQ data. The overall approach is as follows:
1. Load FAQ Data & Structure Metadata
2. Create Embeddings for the FAQ Data using Amazon's Titan Embeddings model, and save these embeddings locally using Chroma.
3. Create a Question Answering (QA) chain which retrieves context based on the embeddings saved in Chroma, serving these as context to the Amazon Titan Express LLM to answer the provided user prompt.
4. Format the response so that source materials can be cited.

This implementation mostly follows these Langchain tutorials:
- https://python.langchain.com/docs/modules/data_connection/document_loaders/json#using-jsonloader

In [1]:
# Define metadata extraction function so we can filter on sections and return deep links as sources
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["section"] = record.get("section")
    metadata["source"] = record.get("deep_link")
    
    return metadata

In [2]:
# Import JSON FAQ File using JSONLoader
from langchain_community.document_loaders import JSONLoader
from pprint import pprint

file_path='../data/processed/faq_data/KU_NSYR.json'

loader = JSONLoader(
    file_path=file_path,
    jq_schema=".[]",
    content_key="answer",
    metadata_func=metadata_func
)

data = loader.load()

In [3]:
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1", region_name="us-east-1"
)
persistDirectory = './data/processed/faq_data/vectordata/KU_NSYR'
vectorstore = Chroma.from_documents(
    documents=data, embedding=embeddings, persist_directory=persistDirectory)
vectorstore.persist()

Run the cells below to re-use already-generated vector embeddings.

In [4]:
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1", region_name="us-east-1"
)

persistDirectory = './data/processed/faq_data/vectordata/KU_NSYR'
# Retrieve and generate answers using relevant FAQs
vectordb = Chroma(persist_directory=persistDirectory,
                  embedding_function=embeddings)

retriever = vectordb.as_retriever()

In [5]:
from utils import CreateInferenceModifier # Import the function from utils.py
from langchain import hub
from langchain_community.llms import Bedrock
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt_template = """Drawing from your knowledge base, answer the question below.
If you don't know the answer from the provided context, tell me that your training materials don't include this information.
Keep the answer as concise and relevant to the question as possible.

Knowledge Base: {context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(prompt_template)


# Define the universal set of modifier parameters
modifiers = {"max_tokens": 4000,
             "temperature": 0.2,
             "top_k": 250,
             "top_p": 1,
             "stop_sequences": ["\n\nHuman"],
             }


llm = Bedrock(model_id="anthropic.claude-v2:1",
              model_kwargs=CreateInferenceModifier("claude", params=modifiers))

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [6]:
# Define a function to extract unique URLs used in the retrieved source materials.
from IPython.display import Markdown, display
def extract_unique_urls(response):
    unique_urls = set()  # Use a set to store unique URLs
    
    # Iterate through each document in the 'context'
    for document in response['context']:
        source_url = document.metadata['source']  # Extract the 'source' URL
        unique_urls.add(source_url)  # Add the URL to the set
    
    # Convert the set of unique URLs to a string
    urls_string = '; '.join(unique_urls)
    
    return urls_string


# Invoke the chain and print the response and sources.
response = rag_chain_with_source.invoke(
    "چی هەنگاوەکانم پێویستە بنێم بۆ داواکردنی پەنابەری لە تورکیا؟")
answer = response["answer"]
prettyResponse = f"## Answer: \n\n {answer} \n\n ## Sources: \n\n {extract_unique_urls(response)}"
Markdown(prettyResponse)

## Answer: 

  بۆ داواکردنی پەنابەری لە تورکیا، ئەم هەنگاوانەت پێویستە:

1. تۆمارکردن لەگەڵ بەڕێوەبەرایەتیی هەرێمی بۆ بەڕێوەبردنی کۆچ و تۆمارکردنی داخوازینامەی پاراستنی نێودەوڵەتی

2. چاوپێکەوتنی تۆمارکردن لەگەڵ بەرپرسان و پاسخدانەوەی پرسیارەکانیان دەربارەی هۆکارەکانی جێهێشتن 

3. چاوپێکەوتنی کەسی بۆ شرۆڤەکردنی هۆکارەکانی جێهێشتن بە تێروتەسەلی 

4. چاوەڕوان بە وەڵامی هەڵسەنگاندنی داخوازینامەکە 

5. ئەگەر وەڵام ئەرێنی بوو، وەرگرتنی بەڵگەنامەی ناسنامەی داخوازیکەری پاراستنی نێودەوڵەتی

6. درێژکردنەوەی ماوەی بەڵگەنامەی ناسنامە بە سەردانکردنی بەڕێوەبەرایەتیی هەرێمی بۆ بەڕێوەبردنی کۆچ  

ئەمانە گرنگترین هەنگاوەکانن بۆ داواکردن و وەرگرتنی پەنابەری لە تورکیا. 

 ## Sources: 

 https://multecihaklari.info/ku/services/labor-market-4/?section=questions&question=0; https://multecihaklari.info/ku/services/registration-and-status-2/?section=questions&question=6; https://multecihaklari.info/ku/services/detention-4/?section=questions&question=0; https://multecihaklari.info/ku/services/legal-assistance-4/?section=questions&question=2

In [7]:
def query_asylum_system(question):
    from IPython.display import Markdown, display
    # Invoke the LLM chain with the provided question
    response = rag_chain_with_source.invoke(question)

    # Extract the answer from the response
    answer = response["answer"]

    # Format the response using Markdown
    pretty_response = f"## Answer: \n\n{answer}\n\n## Sources:\n\n{extract_unique_urls(response)}"
    return pretty_response

In [8]:
Markdown(query_asylum_system(
    "ئایا دەتوانیت لێم بڵێیت چی مافەکانم هەیە وەکو پەنابەرێک لە تورکیا؟"))

## Answer: 

 بەپێی زانیارییەکانی ناو زانیارینامەکەم، ئەمانە مافە سەرەکییەکانت هەیە وەکو پەنابەرێک لە تورکیا:

- مافی داواکردنی پەنابەری و مافی پاراستن لە دیپۆرتکردنەوە
- مافی کارکردن و مافی خوێندن
- مافی تەندروستی و چاودێریی تەندروستی
- مافی دەستڕاگەیشتن بە شوێنی نیشتەجێبوون و خواردن و پارەی گیرفان
- مافی یەکگرتنەوەی خێزان و مافی دامەزراندنی خێزان
- مافی ئازادیی بیروڕا و ئایین و ڕادەربڕین

ئەم مافانە بەشێوەیەکی گشتی لەلایەن یاساکانی تورکیاوە پارێزراون بۆ پەنابەران. بۆ زانیاری زیاتر سەبارەت بە مافەکانت، پێشنیار دەکەم پەیوەندی بە یەکێک لە ڕێکخراوە ماف-پارێزەکان بکەیت.

## Sources:

https://multecihaklari.info/ku/services/unaccompanied-minors-4/?section=questions&question=13; https://multecihaklari.info/ku/services/unaccompanied-minors-4/?section=questions&question=9; https://multecihaklari.info/ku/services/detention-4/?section=questions&question=0; https://multecihaklari.info/ku/services/legal-assistance-4/?section=questions&question=2

In [9]:
Markdown(query_asylum_system("پارەی گیرفان چییە؟ کێ دەتوانێت بەدەستی بهێنێت؟"))

## Answer: 

 <p><span style="font-weight: 400;">لە کۆتایی زانیارییەکەدا هاتووە کە:</span></p>

<p><span style="font-weight: 400;">"هەروەها دوای دەرچوونت بە بێ مۆڵەت لە دامەزراوەیەکدا، ناتوانیت چیدی بەردەوام بیت لە سوودمەندبوون لە مافەکانت وەک خوێندن، دەستڕاگەیشتن بە چاودێری تەندروستی، دەستڕاگەیشتن بە شوێنی نیشتەجێبوون، خواردن، پارەی گیرفان و هتد."</span></p>

<p><span style="font-weight: 400;">لێرەدا "پارەی گیرفان" ئاماژە بە پارەیەک دەکات کە دەدرێت بە منداڵانی بێ سەرپەرشت بۆ یارمەتیدانیان لە ژیان. ئەم پارانە دەدرێن لەلایەن دەزگا حکومییەکانەوە بۆ منداڵانی بێ سەرپەرشت.</span>

<p><span style="font-weight: 400;">بەپێی زانیارییەکانی پێشکەشکراو، تەنیا منداڵانی بێ سەرپەرشت دەتوانن ئەم پارانە وەربگرن. </span></p>

## Sources:

https://multecihaklari.info/ku/services/unaccompanied-minors-4/?section=questions&question=13; https://multecihaklari.info/ku/services/labor-market-4/?section=questions&question=0; https://multecihaklari.info/ku/services/education-4/?section=questions&question=13; https://multecihaklari.info/ku/services/legal-assistance-4/?section=questions&question=2